# sscan as Bluesky plan

## 1D step scans using sscan record

Support the [sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) with a  [bluesky](http://nsls-ii.github.io/bluesky) plan for data acquisition.  Consider the case of [1D step scans using sscan record](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html#HEADING_1-1).

In [ ]:
import asyncio
from collections import deque, OrderedDict
import numpy as np
import time

%matplotlib notebook
from bluesky.utils import install_qt_kicker
install_qt_kicker()

# common IOC prefix to be used
P = "prj:"

In [ ]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH(f"{P}scaler1", name="scaler")
scaler.select_channels(None)

In [ ]:
from ophyd import EpicsMotor
m1 = EpicsMotor(f"{P}m1", name="m1")

In [ ]:
from apstools.synApps_ophyd import userCalcsDevice
calcs = userCalcsDevice(P, name="calcs")

In [ ]:
from apstools.synApps_ophyd import sscanDevice
scans = sscanDevice(P, name="scans")
scans.select_channels()

In [ ]:
from apstools.synApps_ophyd import SaveData
save_data = SaveData(f"{P}saveData_", name="save_data")

In [ ]:
# configure saveData for data collection into MDA files:
        
save_data.file_system.put("/tmp")
save_data.subdirectory.put("saveData")
save_data.base_name.put("sscan1_")
save_data.next_scan_number.put(1)
save_data.comment1.put("testing")
save_data.comment2.put("configured and run from ophyd")

In [ ]:
# configure the sscan record for data collection:

# clear out the weeds
scans.reset()

scan = scans.scan1
scan.number_points.put(6)
scan.positioners.p1.setpoint_pv.put(m1.user_setpoint.pvname)
scan.positioners.p1.readback_pv.put(m1.user_readback.pvname)
scan.positioners.p1.start.put(-1)
scan.positioners.p1.end.put(0)
scan.positioner_delay.put(0.0)
scan.detector_delay.put(0.1)
scan.detectors.d01.input_pv.put(scaler.channels.chan03.s.pvname)
scan.detectors.d02.input_pv.put(scaler.channels.chan02.s.pvname)
scan.triggers.t1.trigger_pv.put(scaler.count.pvname)

# finally, reconfigure
scans.select_channels()

In [ ]:
# make a noisy detector in an EPICS swait record, peak ceneter at 2
from apstools.synApps_ophyd import swait_setup_lorentzian
swait_setup_lorentzian(calcs.calc2, m1, 2)
noisy_det = calcs.calc2.val
noisy_det.kind = "hinted"

In [ ]:
def ophyd_step_scan(motor):
    """step-scan the motor and read the noisy detector"""
    t0 = time.time()
    for p in range(10):
        motor.move(p-3)
        print(
            "%8.3f" % (time.time()-t0), 
            "%8.2f" % motor.position, 
            "%8.4f" % noisy_det.get()
             )
    motor.move(0)
    print("Complete in %.3f seconds" % (time.time()-t0))

# ophyd_step_scan(m1)

--------
## setup Bluesky, databroker, and the RunEngine

In [ ]:
from databroker import Broker
db = Broker.named("mongodb_config")

In [ ]:
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky import SupplementalData

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())
RE.preprocessors.append(SupplementalData())

simple step scan using bluesky plan

In [ ]:
# RE(bp.scan([noisy_det], m1, -5, 5, 11))

------
## Develop the BS plan